In [ ]:
# --- BLOK 1: KURULUM, DRIVE VE KÜTÜPHANELER ---
print("Blok 1 Çalışıyor: Kurulum, Drive Bağlantısı ve Kütüphaneler...")

from google.colab import drive
drive.mount('/content/drive')

import zipfile, os
import time
import json
import io # JPEG sıkıştırması için eklendi

# 1. ZIP dosyasını açma (Eğer 'cifake' klasörü yoksa)
zip_path = "/content/drive/MyDrive/archive (3).zip"
extract_path = "/content/cifake"

if not os.path.exists(extract_path):
    print(f"'{extract_path}' bulunamadı, ZIP açılıyor (5-10 dk sürebilir)...")
    with zipfile.ZipFile(zip_path, 'r') as z:
        z.extractall(extract_path)
else:
    print(f"'{extract_path}' zaten mevcut, ZIP açma adımı atlanıyor.")

# 2. Gerekli kütüphaneyi kurma
!pip install xgboost

# 3. Gerekli tüm kütüphaneleri import etme
import glob, os
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
from torchvision import models
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support, roc_auc_score,
    confusion_matrix, ConfusionMatrixDisplay, roc_curve, auc
)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
import warnings
warnings.filterwarnings("ignore") # Gereksiz uyarıları kapat

print("\n--- Blok 1 Tamamlandı. Kütüphaneler yüklendi. ---")

In [ ]:
# --- BLOK 2: VERİ YOLU BULMA VE DATAFRAME OLUŞTURMA ---
print("Blok 2 Çalışıyor: Veri Yolları Bulunuyor ve DataFrame'ler Oluşturuluyor...")

# 1. Veri setinin ana klasörünü bulma (içinde 'train' ve 'test' olan)
search_root = "/content/cifake"
data_root = None
for dirpath, dirnames, filenames in os.walk(search_root):
    if "train" in dirnames and "test" in dirnames:
        data_root = dirpath
        break
print(f"Bulunan DATA_ROOT (içinde train/test olan): {data_root}")

# 2. Tüm dosya yollarını 'glob' ile bulma
DATA_ROOT = data_root # Bulunan yolu kullan
train_dir = os.path.join(DATA_ROOT, "train")
test_dir  = os.path.join(DATA_ROOT, "test")

train_real = glob.glob(os.path.join(train_dir, "REAL", "*"))
train_fake = glob.glob(os.path.join(train_dir, "FAKE", "*"))
test_real  = glob.glob(os.path.join(test_dir, "REAL", "*"))
test_fake  = glob.glob(os.path.join(test_dir, "FAKE", "*"))

# 3. DataFrame'leri oluşturma (0=REAL, 1=FAKE)
df_train = pd.DataFrame({
    "path": train_real + train_fake,
    "label": [0]*len(train_real) + [1]*len(train_fake)
})
df_test = pd.DataFrame({
    "path": test_real + test_fake,
    "label": [0]*len(test_real) + [1]*len(test_fake)
})

df_train = df_train.sample(frac=1, random_state=42).reset_index(drop=True)
df_test  = df_test.sample(frac=1, random_state=42).reset_index(drop=True)

# 4. Veriyi K-Fold (20k) ve Kilitli Test (20k) olarak bölme
egitim_alt_kume_df, _ = train_test_split(
    df_train,
    train_size=20000,
    random_state=42,
    stratify=df_train["label"]
)
kilitli_test_df = df_test.copy()  # Orijinal test setini kilitliyoruz

print(f"K-Fold için alt küme boyutu: {len(egitim_alt_kume_df)}")
print(f"Kilitli Test Seti boyutu: {len(kilitli_test_df)}")
print(f"Alt küme dağılımı:\n {egitim_alt_kume_df['label'].value_counts(normalize=True)}")

print("\n--- Blok 2 Tamamlandı. Veri setleri (DataFrame) hafızada. ---")

In [ ]:
# --- BLOK 3: ÖZELLİK ÇIKARIMI (ResNet50 @ 224x224) ---
# (A100 GPU ile 5-10 dk sürecek)
print("Blok 3 Çalışıyor: Özellik Çıkarımı (ResNet50 @ 224x224)...")

# 1. Cihazı (GPU) Ayarlama
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Kullanılan Cihaz (MUTLAKA 'cuda' OLMALI): {device}")

# 2. Görüntü Dönüşümleri (ResNet50 için 224x224)
transform_resnet50 = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225])
])

# 3. Özel ImageDataset Sınıfı
class ImageDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df.reset_index(drop=True)
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        path = self.df.loc[idx, "path"]
        label = int(self.df.loc[idx, "label"])
        try:
            img = Image.open(path).convert("RGB")
            img = self.transform(img)
            return img, label
        except Exception as e:
            return torch.zeros((3, 224, 224)), -1 # Bozuksa -1 label'ı ver

def collate_fn_safe(batch):
    batch = list(filter(lambda x: x[1] != -1, batch)) # -1 label'lıları atla
    if len(batch) == 0:
        return torch.Tensor(), torch.Tensor()
    return torch.utils.data.dataloader.default_collate(batch)

# 4. ResNet50 Modelini Özellik Çıkarıcı Olarak Yükleme
resnet = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
resnet.fc = nn.Identity() # Son katmanı (sınıflandırıcıyı) kaldır
resnet = resnet.to(device)
resnet.eval()

# 5. Özellik Çıkarıcı Fonksiyon
@torch.no_grad()
def extract_features(df, transform_to_use, batch_size=64):
    dataset = ImageDataset(df, transform_to_use)
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn_safe)
    feats, labels = [], []

    for imgs, labs in tqdm(loader, desc=f"Özellik Çıkarımı ({len(df)} görsel)"):
        if imgs.shape[0] == 0: continue
        imgs = imgs.to(device)
        f = resnet(imgs) # [B, 2048]
        feats.append(f.cpu().numpy())
        labels.append(labs.numpy())

    return np.concatenate(feats, axis=0), np.concatenate(labels, axis=0)

# 6. Fonksiyonu Çalıştırma (ResNet50 dönüşümü ile)
print("K-Fold Eğitim Seti için özellik çıkarımı başlıyor...")
X_train_all, y_train_all = extract_features(egitim_alt_kume_df, transform_resnet50)

print("Kilitli Test Seti için özellik çıkarımı başlıyor...")
X_test_locked, y_test_locked = extract_features(kilitli_test_df, transform_resnet50)

print(f"Train feature shape: {X_train_all.shape}, Train label shape: {y_train_all.shape}")
print(f"Test feature shape: {X_test_locked.shape}, Test label shape: {y_test_locked.shape}")
print("\n--- Blok 3 Tamamlandı. 'resnet', 'X_train_all' ve 'X_test_locked' hafızada. ---")

In [ ]:
# --- BLOK 4: ŞAMPİYON MODELİ YENİDEN EĞİTME (HIZLI YOL) ---
# (K-Fold DEĞİL, tek bir eğitim. ~15-20 dk sürecek)
print("Blok 4 Çalışıyor: Şampiyon Model (SVM_RBF) Yeniden Eğitiliyor...")

# 1. Şampiyon modelimizi (SVM_RBF) tanımlıyoruz
# (NOT: Eğer 'best_name' XGBoost çıktıysa, buradaki 'best_model'i ona göre değiştir)
best_name = "SVM_RBF"
best_model = make_pipeline(StandardScaler(), SVC(kernel="rbf", probability=True))

print(f"Şampiyon model '{best_name}' hafızaya yükleniyor...")
print(f"'{best_name}' modeli TÜM (20,000) eğitim verisiyle yeniden eğitiliyor...")
print("(Bu işlem 15-20 dakika sürebilir, lütfen bekleyin...)")

start_train_time = time.time()
# 2. Modeli TÜM X_train_all verisiyle EĞİT
best_model.fit(X_train_all, y_train_all)
end_train_time = time.time()

print(f"Eğitim Tamamlandı! (Süre: {(end_train_time - start_train_time)/60:.2f} dakika)")
print("\n--- Blok 4 Tamamlandı. 'best_model' (Şampiyon) artık hafızada. ---")

In [ ]:
# --- BLOK 5: DAYANIKLILIK (ROBUSTNESS) TESTİ İÇİN HAZIRLIK ---
print("Blok 5 Çalışıyor: 'Bozukluk' (Distortion) Fonksiyonları Hazırlanıyor...")

# --- 1. Yeni 'Bozukluk' (Distortion) Transform'ları Tanımlama ---

# A) Gaussian Gürültü (Karıncalanma) Ekleme
class AddGaussianNoise(object):
    def __init__(self, mean=0., std=1.):
        self.std = std
        self.mean = mean
    def __call__(self, tensor):
        return tensor + torch.randn(tensor.size()) * self.std + self.mean
    def __repr__(self):
        return self.__class__.__name__ + f'(mean={self.mean}, std={self.std})'

# B) JPEG Sıkıştırması (Instagram/Twitter Simülasyonu)
class AddJpegCompression(object):
    def __init__(self, quality=50):
        self.quality = quality
    def __call__(self, img_tensor):
        img = T.ToPILImage()(img_tensor)
        buffer = io.BytesIO()
        img.save(buffer, "JPEG", quality=self.quality)
        img_jpeg = Image.open(buffer)
        return T.ToTensor()(img_jpeg)
    def __repr__(self):
        return self.__class__.__name__ + f'(quality={self.quality})'

# --- 2. 'Bozukluk' İçin Yeni Dönüşüm Pipelineları ---
transform_jpeg = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(), # Önce Tensöre çevir
    AddJpegCompression(quality=50), # JPEG Sıkıştırması uygula (boz)
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalize et
])
transform_noise = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]), # ÖNCE Normalize et
    AddGaussianNoise(mean=0., std=0.1) # Sonra gürültü ekle
])
transform_blur = T.Compose([
    T.Resize((224, 224)),
    T.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5.0)), # Bulanıklık ekle
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # Normalize et
])

# --- 3. Bozuk Veri İçin 'ImageDataset' Sınıfı ---
class RobustnessImageDataset(Dataset):
    def __init__(self, df, transform):
        self.df = df.reset_index(drop=True)
        self.transform = transform
    def __len__(self):
        return len(self.df)
    def __getitem__(self, idx):
        path = self.df.loc[idx, "path"]
        label = int(self.df.loc[idx, "label"])
        try:
            img = Image.open(path).convert("RGB")
            img = self.transform(img) # VERİLEN 'BOZUK' TRANSFORMU UYGULA
            return img, label
        except Exception as e:
            return torch.zeros((3, 224, 224)), -1 # Bozuksa -1 label'ı ver

print("\n--- Blok 5 Tamamlandı. 'Bozuk' veri yükleyicileri hazır. ---")

In [ ]:
# --- BLOK 6: DAYANIKLILIK (ROBUSTNESS) TESTİ ÇALIŞTIRMA VE FİNAL ---
print("Blok 6 Çalışıyor: FİNAL DAYANIKLILIK TESTLERİ BAŞLIYOR...")

# 1. Test Fonksiyonu (Resimleri bozar, ResNet'ten geçirir, SVM'e sorar)
@torch.no_grad() # Bu testte gradyan (eğitim) yok
def test_robustness(ml_model, feature_extractor, df, transform_to_use):

    print(f"\nTest başlıyor (Transform: {transform_to_use.__repr__()})...")

    # 1. Bozuk veriyi yükle
    dataset = RobustnessImageDataset(df, transform_to_use)
    loader = DataLoader(dataset, batch_size=64, shuffle=False, collate_fn=collate_fn_safe)

    feature_extractor.eval() # ResNet50'yi 'eval' moduna al

    all_distorted_features = [] # Bozuk özellikleri topla
    all_labels = []

    # 2. ResNet50 (GPU) ile 'bozuk özellikleri' çıkar
    for imgs, labs in tqdm(loader, desc="Bozuk Özellikler Çıkarılıyor"):
        if imgs.shape[0] == 0: continue
        imgs = imgs.to(device) # Resimleri GPU'ya yolla
        f = feature_extractor(imgs) # ResNet50'den özellikleri al
        all_distorted_features.append(f.cpu().numpy()) # Özellikleri CPU'ya geri al
        all_labels.append(labs.numpy())

    X_distorted = np.concatenate(all_distorted_features)
    y_true = np.concatenate(all_labels)

    # 3. SVM/XGBoost (CPU) ile tahmin yap
    print(f"'{best_name}' modeli ile {len(X_distorted)} adet bozuk özellik üzerinde tahmin yapılıyor...")
    y_pred = ml_model.predict(X_distorted)

    # 4. Metrikleri hesapla
    acc = accuracy_score(y_true, y_pred)
    prec, rec, f1, _ = precision_recall_fscore_support(y_true, y_pred, average="binary", zero_division=0)

    print(f"Test Bitti: F1-Skoru = {f1*100:.2f}%")
    return acc, prec, rec, f1

# 2. Testleri Çalıştırma
robust_results = {}
try:
    # Test 1: JPEG Sıkıştırma
    acc_jpeg, pre_jpeg, rec_jpeg, f1_jpeg = test_robustness(
        best_model, resnet, kilitli_test_df, transform_jpeg
    )
    robust_results["Sıkıştırılmış JPEG (%50)"] = f1_jpeg

    # Test 2: Gürültü Ekleme
    acc_noise, pre_noise, rec_noise, f1_noise = test_robustness(
        best_model, resnet, kilitli_test_df, transform_noise
    )
    robust_results["Gaussian Gürültü (std=0.1)"] = f1_noise

    # Test 3: Bulanıklık Ekleme
    acc_blur, pre_blur, rec_blur, f1_blur = test_robustness(
        best_model, resnet, kilitli_test_df, transform_blur
    )
    robust_results["Gaussian Bulanıklık"] = f1_blur

    # 3. FİNAL RAPORUNU OLUŞTURMA
    print("\n\n===== HİBRİT MODEL DAYANIKLILIK (ROBUSTNESS) ANALİZİ =====")
    print(f"Test Edilen Şampiyon Model: {best_name} (Hibrit ML)\n")

    # Temiz veri sonucunu da hesaplayalım (ne olur ne olmaz)
    print("Temiz veri skoru yeniden hesaplanıyor (karşılaştırma için)...")
    y_pred_clean = best_model.predict(X_test_locked)
    clean_f1 = f1_score(y_test_locked, y_pred_clean, average="binary")

    print(f"Karşılaştırma (F1-Skoru %):")
    print("----------------------------------------------------------")
    print(f"  Temiz Veri Seti (Kilitli Test): \t{clean_f1*100:.2f}%")
    print("----------------------------------------------------------")
    for test_name, f1_score in robust_results.items():
        print(f"  {test_name}: \t{f1_score*100:.2f}%")
    print("----------------------------------------------------------")

    # Bu sonuçları da bir dosyaya kaydet
    with open("Hibrit_Dayaniklilik_Raporu.txt", "w") as f:
        f.write(f"===== HİBRİT MODEL DAYANIKLILIK (ROBUSTNESS) ANALİZİ =====\n")
        f.write(f"Test Edilen Model: {best_name} (Hibrit ML)\n\n")
        f.write(f"Karşılaştırma (F1-Skoru %):\n")
        f.write("----------------------------------------------------------\n")
        f.write(f"  Temiz Veri Seti (Kilitli Test): \t{clean_f1*100:.2f}%\n")
        f.write("----------------------------------------------------------\n")
        for test_name, f1_score in robust_results.items():
            f.write(f"  {test_name}: \t{f1_score*100:.2f}%\n")
        f.write("----------------------------------------------------------\n")
    print("\n'Hibrit_Dayaniklilik_Raporu.txt' dosyası da kaydedildi.")


except NameError as e:
    print(f"\n--- HATA --- ({e})")
    print("Görünüşe göre Blok 3 veya 4'te bir sorun oldu ve 'best_model' veya 'resnet' hafızaya yüklenemedi.")
    print("Lütfen Blok 1'den itibaren tekrar çalıştırmayı deneyin.")
except Exception as e:
    print(f"\nBaşka bir hata oluştu: {e}")

print("\n--- Blok 6 Tamamlandı. Özgünlük (Dayanıklılık) analizi bitti. ---")